# Day 4

## Tokenizing with code

In [8]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4.1-mini")

tokens = encoding.encode("Hi my name is Ryan and I like zafron pie importantlyness")

In [9]:
tokens

[12194,
 922,
 1308,
 382,
 28706,
 326,
 357,
 1299,
 579,
 1553,
 2963,
 5148,
 33766,
 2816]

In [10]:
for token_id in tokens:
    token_text = encoding.decode([token_id])
    print(f"{token_id} = {token_text}")

12194 = Hi
922 =  my
1308 =  name
382 =  is
28706 =  Ryan
326 =  and
357 =  I
1299 =  like
579 =  z
1553 = af
2963 = ron
5148 =  pie
33766 =  importantly
2816 = ness


In [4]:
encoding.decode([326])

' and'

# And another topic!

### The Illusion of "memory"

Many of you will know this already. But for those that don't -- this might be an "AHA" moment!

In [12]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


### You should be very comfortable with what the next cell is doing!

_I'm creating a new instance of the OpenAI Python Client library, a lightweight wrapper around making HTTP calls to an endpoint for calling the GPT LLM, or other LLM providers_

In [13]:
from openai import OpenAI

openai = OpenAI()

### A message to OpenAI is a list of dicts

In [21]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Hi! I'm Ryan! can you remember my birthday? 8th of May"}
    ]

In [22]:

OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

response = ollama.chat.completions.create(model="llama3.2", messages=messages)
# response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)


response.choices[0].message.content

"Hello Ryan! Nice to meet you. I'm happy to help, but unfortunately, I'm a large language model, I don't have personal memories or keep track of individual users' birthdays. Each time you interact with me, it's a new conversation.\n\nHowever, I can certainly remember (temporarily) that your birthday is on May 8th! Would you like to chat about something in particular or just shoot the breeze?"

### OK let's now ask a follow-up question

In [23]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What's my birthday and what is my name?"}
    ]

In [24]:
#response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
response = ollama.chat.completions.create(model="llama3.2", messages=messages)
response.choices[0].message.content

"I don't have any information about you. I'm a text-based AI assistant, and our conversation just started. You've told me nothing about yourself. Would you like to share some details about your identity, or would you rather change the topic?"

### Wait, wha??

We just told you!

What's going on??

Here's the thing: every call to an LLM is completely STATELESS. It's a totally new call, every single time. As AI engineers, it's OUR JOB to devise techniques to give the impression that the LLM has a "memory".

In [31]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Hi! I'm Ryan!"},
    {"role": "assistant", "content": "Hi Ryan! How can I assist you today?"},
    {"role": "user", "content": "What's my name? can you say my name?"}
    ]

In [32]:
# response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
response = ollama.chat.completions.create(model="llama3.2", messages=messages)
response.choices[0].message.content

"Your name is... Ryan. How's that, Ryan?"

## To recap

With apologies if this is obvious to you - but it's still good to reinforce:

1. Every call to an LLM is stateless
2. We pass in the entire conversation so far in the input prompt, every time
3. This gives the illusion that the LLM has memory - it apparently keeps the context of the conversation
4. But this is a trick; it's a by-product of providing the entire conversation, every time
5. An LLM just predicts the most likely next tokens in the sequence; if that sequence contains "My name is Ed" and later "What's my name?" then it will predict.. Ed!

The ChatGPT product uses exactly this trick - every time you send a message, it's the entire conversation that gets passed in.

"Does that mean we have to pay extra each time for all the conversation so far"

For sure it does. And that's what we WANT. We want the LLM to predict the next tokens in the sequence, looking back on the entire conversation. We want that compute to happen, so we need to pay the electricity bill for it!

